# Libraries Import

In [ ]:
import json
import pandas as pd

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from collections import Counter
from string import punctuation

In [ ]:
import numpy as np

In [ ]:
!pip install pyprind

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyprind

In [ ]:
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

In [ ]:
from tqdm import tqdm
from scipy import stats

# Mount Google Drive

In [ ]:
from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
f = open('drive/MyDrive/memento_train_data.json')
data = json.load(f)

In [ ]:
for key in data[0]:
  print(key, data[0][key])

filename adult-female-singing_peeks-www_k_to_keek_2Qwueab_25.mp4
url https://data.csail.mit.edu/soundnet/actions3/adult+female+singing/peeks-www_k_to_keek_2Qwueab_25.mp4
mem_score 0.9891514478670866
alpha -0.0003866407543994376
audio_description indoors you see a woman with red hair held up in a bun and piercings on either side of her lip she's looking sad
captions ['A brown hair girl with lip piercings mumbles something then looks away.', 'A brown hair girl that looks sad opens her mouth and says something before turning away.', 'A girl with brown hair and lip rings looks away from the camera.', 'A young woman wearing a striped shirt, who has two piercings in her lower lip, looks distressed.', 'A girl staring at the camera with earnings on its mouth']
action_labels ['holding', 'looking', 'adult+female+singing', 'singing']


In [ ]:
train_scores = pd.read_csv('drive/MyDrive/train_scores.csv')

In [ ]:
train_scores.head()

,video_id,video_url,annotations_short_term,scores_short_term,decay_alpha
0,17,https://data.csail.mit.edu/soundnet/actions3/a...,92,0.989151,-0.000387
1,3628,https://data.csail.mit.edu/soundnet/actions3/a...,91,0.913391,0.000410
2,6515,https://data.csail.mit.edu/soundnet/actions3/a...,91,0.932910,-0.000518
3,471,https://data.csail.mit.edu/soundnet/actions3/a...,89,0.966062,-0.000108
4,4171,https://data.csail.mit.edu/soundnet/actions3/a...,91,0.944808,-0.000662


In [ ]:
train_descriptions = pd.read_csv('drive/MyDrive/train_text_descriptions.csv')

In [ ]:
train_descriptions.head()

,video_id,description_0,description_1,description_2,description_3,description_4
0,17,A brown hair girl with lip piercings mumbles s...,A brown hair girl that looks sad opens her mou...,A girl with brown hair and lip rings looks awa...,"A young woman wearing a striped shirt, who has...",A girl staring at the camera with earnings on ...
1,3628,A woman is standing and singing under a ceilin...,A woman is swaying in time as she sings to the...,A woman is standing in a room and singing a song.,A woman in a blue sweater sings while facing t...,A lady is in a room with a fan above her head ...
2,6515,A person wearing a white sleeveless shirt is c...,A fat woman claps and hits the table while spe...,A larger woman in a white tank top claps her h...,Someone in a white v neck tee shirt is shown c...,A brunette sings and claps before in front of ...
3,471,"A woman leans against a red, decorated wall an...",A woman with a black and white top and peach s...,A black lady is singing and dancing in front o...,A woman in black and white sings and dances in...,A black lady is dancing in front of a red wall.
4,4171,Two overweight women sway next to each other a...,Two ladies are standing in an open room while ...,Two morbidly obese women dance and sing in a r...,Two white women standing and singing while mov...,two fatties stand around and sing while swayin...


In [ ]:
len(train_descriptions)

7000

# Clean Captions

**Only First Description**

In [ ]:
cleaned_captions = []

pos_list = []
neg_list = []

for i in range(0, 7000):
    local_pos_weight = 0
    local_neg_weight = 0
    text = re.sub('[^a-zA-Z]', ' ', train_descriptions['description_0'][i])
    text = text.lower()
    text = text.split()
    text = [word for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    cleaned_captions.append(text)

**Concatenate All Descriptions**

In [ ]:
cleaned_captions = []

pos_list = []
neg_list = []

for i in tqdm(range(0, 7000)):
    local_pos_weight = 0
    local_neg_weight = 0
    text = re.sub('[^a-zA-Z]', ' ', train_descriptions['description_0'][i]+" "+train_descriptions['description_1'][i]+" "+train_descriptions['description_2'][i]+" "+train_descriptions['description_3'][i]+" "+train_descriptions['description_4'][i])
    text = text.lower()
    text = text.split()
    text = [word for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    cleaned_captions.append(text)

100%|██████████| 7000/7000 [01:13<00:00, 95.19it/s]


In [ ]:
cleaned_captions

['brown hair girl lip piercings mumbles something looks away brown hair girl looks sad opens mouth says something turning away girl brown hair lip rings looks away camera young woman wearing striped shirt two piercings lower lip looks distressed girl staring camera earnings mouth',
 'woman standing singing ceiling fan white room woman swaying time sings camera woman standing room singing song woman blue sweater sings facing camera lady room fan head singing',
 'person wearing white sleeveless shirt clapping saying words time fat woman claps hits table speaking larger woman white tank top claps hands someone white v neck tee shirt shown clapping hands brunette sings claps front camera',
 'woman leans red decorated wall poses camera woman black white top peach skirt dance laughs black lady singing dancing front red wall woman black white sings dances front red wall black lady dancing front red wall',
 'two overweight women sway next one sings two ladies standing open room singing song to

In [ ]:
len(cleaned_captions)

7000

# TF-IDF Vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()

In [ ]:
caption_features = tf.fit_transform(cleaned_captions).toarray()

In [ ]:
caption_features.shape

(7000, 10021)

In [ ]:
caption_features_train = caption_features

In [ ]:
caption_features_train.shape

(7000, 10021)

In [ ]:
mem_scores = []

for item in data:
  mem_scores.append(item['mem_score'])

# X and Y Datapoints

In [ ]:
X = caption_features_train
y = mem_scores

In [ ]:
len(X)

7000

In [ ]:
len(y)

7000

In [ ]:
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [ ]:
print('X_train', X_train.shape)
print('X_test', X_test.shape)
print('Y_train', y_train.shape)
print('Y_test', y_test.shape)

X_train (5600, 10021)
X_test (1400, 10021)
Y_train (5600,)
Y_test (1400,)


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor(verbose=True)
RFregressor.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: ignored

In [ ]:
y_pred = RFregressor.predict(X_test)

In [ ]:
stats.spearmanr(y_pred, y_test)

# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
LMregressor = LinearRegression()
LMregressor.fit(X_train, y_train)

y_pred = LMregressor.predict(X_test)
print(stats.spearmanr(y_pred, y_test))

SpearmanrResult(correlation=0.22776654721445547, pvalue=6.227932212072459e-18)


# Decision Tree Regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train, y_train)

y_pred = DTregressor.predict(X_test)
print(stats.spearmanr(y_pred, y_test))

SpearmanrResult(correlation=0.2845709396482434, pvalue=1.719533313930546e-27)


# C3D


Reading Captions

In [ ]:
des = pd.read_csv('drive/MyDrive/train_text_descriptions.csv')

In [ ]:
des.head()

,video_id,description_0,description_1,description_2,description_3,description_4
0,17,A brown hair girl with lip piercings mumbles s...,A brown hair girl that looks sad opens her mou...,A girl with brown hair and lip rings looks awa...,"A young woman wearing a striped shirt, who has...",A girl staring at the camera with earnings on ...
1,3628,A woman is standing and singing under a ceilin...,A woman is swaying in time as she sings to the...,A woman is standing in a room and singing a song.,A woman in a blue sweater sings while facing t...,A lady is in a room with a fan above her head ...
2,6515,A person wearing a white sleeveless shirt is c...,A fat woman claps and hits the table while spe...,A larger woman in a white tank top claps her h...,Someone in a white v neck tee shirt is shown c...,A brunette sings and claps before in front of ...
3,471,"A woman leans against a red, decorated wall an...",A woman with a black and white top and peach s...,A black lady is singing and dancing in front o...,A woman in black and white sings and dances in...,A black lady is dancing in front of a red wall.
4,4171,Two overweight women sway next to each other a...,Two ladies are standing in an open room while ...,Two morbidly obese women dance and sing in a r...,Two white women standing and singing while mov...,two fatties stand around and sing while swayin...


In [ ]:
des.shape

(7000, 6)

In [ ]:
caption_features[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
df_cap = pd.DataFrame(columns=['video', 'caption'])

index = 0

for c, v in des.iterrows():
  df_cap.loc[index] = [v['video_id']] + [caption_features[index]]
  index += 1


In [ ]:
df_cap

,video,caption
0,17,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,3628,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,6515,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,471,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4171,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
6995,7054,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6996,4248,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6997,2031,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6998,8158,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
X_seq = pd.DataFrame(df_cap.caption.apply(pd.Series))

In [ ]:
df_seq = pd.DataFrame(X_seq)    #create dataframe with the sequences
df_seq = pd.concat([df_cap, df_seq], axis = 1)   #merging sequnces dataframe and the captions dataframe to I can line up video name to each sequence
df_seq = df_seq.drop("caption", axis=1)  #removing column with word version of caption so we have a dataframe where the integer sequences are algined w

In [ ]:
df_seq

,video,0,1,2,3,4,5,6,7,8,...,10011,10012,10013,10014,10015,10016,10017,10018,10019,10020
0,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3628,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,7054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6996,4248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6997,2031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6998,8158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
labels = pd.DataFrame(columns=['video', 'mem_scores'])

index = 0

for c, v in train_scores.iterrows():
  labels.loc[index] = [str(v['video_id'])] + [v['scores_short_term']]
  index += 1

In [ ]:
labels.head()

,video,mem_scores
0,17,0.989151
1,3628,0.913391
2,6515,0.932910
3,471,0.966062
4,4171,0.944808


In [ ]:
def read_C3D(fname):   #function to read c3d vectors into dataframe
    C3D = pd.read_csv(fname)
    # print(C3D.columns.values)
    # print(C3D.C3D)
    return C3D.columns.values

In [ ]:
import os

print((os.listdir('drive/MyDrive/C3D/C3D')))

# for filename in os.listdir('drive/MyDrive/C3D/C3D'):
#   print(filename[:5]) 

# c3d_features = pd.DataFrame({'video': filename[:5] for filename in os.listdir('drive/MyDrive/C3D/C3D'), 'C3D': [read_C3D('drive/MyDrive/C3D/C3D'+filename) for filename in os.listdir('drive/MyDrive/C3D/C3D')]})

['00001.mp4.csv', '00005.mp4.csv', '00007.mp4.csv', '00008.mp4.csv', '00009.mp4.csv', '00010.mp4.csv', '00011.mp4.csv', '00013.mp4.csv', '00014.mp4.csv', '00015.mp4.csv', '00016.mp4.csv', '00017.mp4.csv', '00018.mp4.csv', '00019.mp4.csv', '00020.mp4.csv', '00021.mp4.csv', '00022.mp4.csv', '00023.mp4.csv', '00024.mp4.csv', '00026.mp4.csv', '00028.mp4.csv', '00029.mp4.csv', '00030.mp4.csv', '00031.mp4.csv', '00034.mp4.csv', '00036.mp4.csv', '00037.mp4.csv', '00038.mp4.csv', '00041.mp4.csv', '00042.mp4.csv', '00043.mp4.csv', '00044.mp4.csv', '00045.mp4.csv', '00047.mp4.csv', '00048.mp4.csv', '00049.mp4.csv', '00050.mp4.csv', '00051.mp4.csv', '00053.mp4.csv', '00054.mp4.csv', '00056.mp4.csv', '00058.mp4.csv', '00059.mp4.csv', '00060.mp4.csv', '00062.mp4.csv', '00063.mp4.csv', '00066.mp4.csv', '00067.mp4.csv', '00068.mp4.csv', '00069.mp4.csv', '00070.mp4.csv', '00071.mp4.csv', '00072.mp4.csv', '00073.mp4.csv', '00075.mp4.csv', '00074.mp4.csv', '00076.mp4.csv', '00077.mp4.csv', '00078.mp4.cs

In [ ]:
c3d_vals = [read_C3D('drive/MyDrive/C3D/C3D/'+filename) for filename in os.listdir('drive/MyDrive/C3D/C3D')]

In [ ]:
c3d_vals

[array(['0.189100', '0.511600', '0.171700', ..., '0.000000.1889',
        '0.670600', '0.139200'], dtype=object),
 array(['0.000000', '0.419400', '0.000000.1', ..., '0.000000.2430',
        '0.000000.2431', '0.000000.2432'], dtype=object),
 array(['0.938200', '0.000000', '0.000000.1', ..., '0.000000.2773',
        '0.000000.2774', '0.000000.2775'], dtype=object),
 array(['0.077700', '0.938400', '0.395200', ..., '0.000000.2646',
        '0.791200', '0.000000.2647'], dtype=object),
 array(['1.570100', '0.000000', '1.245100', ..., '1.209300',
        '0.000000.1992', '1.042800'], dtype=object),
 array(['0.965900', '0.000000', '0.076900', ..., '0.000000.2089',
        '0.344800', '0.000000.2090'], dtype=object),
 array(['1.720200', '0.750600', '0.000000', ..., '0.229300',
        '0.000000.1492', '0.389600'], dtype=object),
 array(['0.000000', '0.000000.1', '1.028500', ..., '0.000000.2056',
        '2.070900', '0.304600'], dtype=object),
 array(['0.000000', '0.152200', '0.000000.1', ..., '

In [ ]:
from tqdm import tqdm

In [ ]:
c3d = []
# vc = 0

for carr in tqdm(c3d_vals):
  temp = []
  # count = 0
  for num in carr:
    if num.count('.') == 2:
      # print(num, end=" ")
      temp.append(float(0))
      # print("Problem in feature", count, "of video", vc)
      # count += 1
  # print(count)
  # vc += 1
    else:
      temp.append(float(num))
  c3d.append(temp)
# print(vc)

2057
2516
2840
2702
2152
2246
1734
2182
2338
2790
2962
3130
2496
2445
3083
2785
2307
2369
2715
1763
2634
2248
2879
2521
2958
2133
3134
3172
3189
2543
2949
1553
1756
2677
3301
2976
2404
1555
3079
1912
3297
2712
2297
2766
2608
2240
2872
2491
2567
3001
1867
2338
2141
2497
2071
2831
2746
3002
2680
1962
2865
3228
1604
2627
1728
2855
2134
2564
2495
3105
2544
2010
2897
2728
2115
2545
3017
2088
2939
2596
2711
2783
1978
2678
2656
3004
2534
3069
2227
3006
2902
2677
2903
3054
2473
2884
2647
2805
2819
3300
2000
2811
2158
2143
2364
2161
2333
2447
2737
2639
2904
1983
2012
2307
1963
1702
2790
2808
2457
3081
2594
3067
2643
1749
2909
2888
2721
2636
2465
2758
2236
2175
2698
3222
2542
3372
2937
2101
3216
2277
2847
2347
3187
1965
2272
2828
2622
2054
2441
2170
2293
2250
2563
1700
1925
2703
3106
2071
2088
2447
2817
2774
2574
2242
2430
2657
2879
1861
2305
2715
2323
2448
172


In [ ]:
# vid_nos = ["00001", "00005", "00007", "00008", "00009", "00010", "00011", "00013", "00014", "00015", "00016", "00017", "00018", "00019", "00020", "00021", "00022", "00023", "00024", "00026"]

# test = ["00001"]

c3d_features = pd.DataFrame({'video': [filename[:5] for filename in os.listdir('drive/MyDrive/C3D/C3D')], 'C3D': c3d})

In [ ]:
c3d_features.head()

,video,C3D
0,00001,"[0.1891, 0.5116, 0.1717, 0.6851, 0.3309, 0.0, ..."
1,00005,"[0.0, 0.4194, 0.0, 0.2765, 0.0, 0.0, 0.0, 0.0,..."
2,00007,"[0.9382, 0.0, 0.0, 0.9751, 0.0, 0.4528, 0.0, 0..."
3,00008,"[0.0777, 0.9384, 0.3952, 0.0, 0.0, 0.0, 0.0, 1..."
4,00009,"[1.5701, 0.0, 1.2451, 0.5674, 1.2224, 0.0, 0.0..."


In [ ]:
c3d_features

,video,C3D
0,00001,"[0.1891, 0.5116, 0.1717, 0.6851, 0.3309, 0.0, ..."
1,00005,"[0.0, 0.4194, 0.0, 0.2765, 0.0, 0.0, 0.0, 0.0,..."
2,00007,"[0.9382, 0.0, 0.0, 0.9751, 0.0, 0.4528, 0.0, 0..."
3,00008,"[0.0777, 0.9384, 0.3952, 0.0, 0.0, 0.0, 0.0, 1..."
4,00009,"[1.5701, 0.0, 1.2451, 0.5674, 1.2224, 0.0, 0.0..."
...,...,...
167,00247,"[0.9274, 0.4696, 0.0, 0.0, 0.0, 0.1226, 1.1528..."
168,00250,"[0.3749, 0.0, 0.0, 0.4142, 1.8517, 0.0, 0.0323..."
169,00251,"[0.1518, 0.0, 0.0, 0.0, 0.4617, 0.4079, 0.0, 0..."
170,00252,"[0.0, 0.0, 0.0, 0.0101, 0.0, 0.0, 0.0, 0.5485,..."


In [ ]:
features1 = pd.DataFrame(c3d_features.C3D.apply(pd.Series))

In [ ]:
features1

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,0.1891,0.5116,0.1717,0.6851,0.3309,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.1603,0.3077,0.2826,0.0000,0.0000,0.5060,0.6045,0.0000,0.6706,0.1392
1,0.0000,0.4194,0.0000,0.2765,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.1373,1.1186,0.4230,1.2730,0.6045,0.0000,0.0000,0.0000,0.0000
2,0.9382,0.0000,0.0000,0.9751,0.0000,0.4528,0.0000,0.0000,0.2808,0.0000,...,0.0000,0.0000,0.0000,0.4547,0.0979,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.0777,0.9384,0.3952,0.0000,0.0000,0.0000,0.0000,1.2670,1.0444,0.0000,...,0.3379,0.2252,0.4315,0.0759,2.4016,1.0923,0.0000,0.0000,0.7912,0.0000
4,1.5701,0.0000,1.2451,0.5674,1.2224,0.0000,0.0000,0.0000,0.1774,0.9317,...,0.2465,0.0000,1.6743,0.0000,0.0409,0.8993,0.0000,1.2093,0.0000,1.0428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,0.9274,0.4696,0.0000,0.0000,0.0000,0.1226,1.1528,0.0000,0.2090,0.0000,...,0.0000,0.0000,1.0360,1.2922,0.4502,0.0000,0.0000,0.0000,0.0000,0.1357
168,0.3749,0.0000,0.0000,0.4142,1.8517,0.0000,0.0323,0.1412,0.0000,0.0000,...,0.1832,0.0000,0.0000,0.0130,0.0000,0.0000,0.2867,0.0000,0.0000,0.0000
169,0.1518,0.0000,0.0000,0.0000,0.4617,0.4079,0.0000,0.0000,0.0851,0.0000,...,0.8342,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.2120,0.0000
170,0.0000,0.0000,0.0000,0.0101,0.0000,0.0000,0.0000,0.5485,1.0947,1.4470,...,0.0000,0.6041,0.0242,0.0000,0.0747,0.3986,0.0000,0.0000,0.0000,0.0000


In [ ]:
features2 = pd.concat([c3d_features, features1], axis = 1)  #merges features and features1 (splits out into columns)
features2 = features2.drop("C3D", axis=1) 

In [ ]:
features2

,video,0,1,2,3,4,5,6,7,8,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,00001,0.1891,0.5116,0.1717,0.6851,0.3309,0.0000,0.0000,0.0000,0.0000,...,0.1603,0.3077,0.2826,0.0000,0.0000,0.5060,0.6045,0.0000,0.6706,0.1392
1,00005,0.0000,0.4194,0.0000,0.2765,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.1373,1.1186,0.4230,1.2730,0.6045,0.0000,0.0000,0.0000,0.0000
2,00007,0.9382,0.0000,0.0000,0.9751,0.0000,0.4528,0.0000,0.0000,0.2808,...,0.0000,0.0000,0.0000,0.4547,0.0979,0.0000,0.0000,0.0000,0.0000,0.0000
3,00008,0.0777,0.9384,0.3952,0.0000,0.0000,0.0000,0.0000,1.2670,1.0444,...,0.3379,0.2252,0.4315,0.0759,2.4016,1.0923,0.0000,0.0000,0.7912,0.0000
4,00009,1.5701,0.0000,1.2451,0.5674,1.2224,0.0000,0.0000,0.0000,0.1774,...,0.2465,0.0000,1.6743,0.0000,0.0409,0.8993,0.0000,1.2093,0.0000,1.0428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,00247,0.9274,0.4696,0.0000,0.0000,0.0000,0.1226,1.1528,0.0000,0.2090,...,0.0000,0.0000,1.0360,1.2922,0.4502,0.0000,0.0000,0.0000,0.0000,0.1357
168,00250,0.3749,0.0000,0.0000,0.4142,1.8517,0.0000,0.0323,0.1412,0.0000,...,0.1832,0.0000,0.0000,0.0130,0.0000,0.0000,0.2867,0.0000,0.0000,0.0000
169,00251,0.1518,0.0000,0.0000,0.0000,0.4617,0.4079,0.0000,0.0000,0.0851,...,0.8342,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.2120,0.0000
170,00252,0.0000,0.0000,0.0000,0.0101,0.0000,0.0000,0.0000,0.5485,1.0947,...,0.0000,0.6041,0.0242,0.0000,0.0747,0.3986,0.0000,0.0000,0.0000,0.0000


In [ ]:
features2.loc[range(172), 'video'] = [int(r['video']) for i, r in features2.iterrows()] 

In [ ]:
features2

,video,0,1,2,3,4,5,6,7,8,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,1,0.1891,0.5116,0.1717,0.6851,0.3309,0.0000,0.0000,0.0000,0.0000,...,0.1603,0.3077,0.2826,0.0000,0.0000,0.5060,0.6045,0.0000,0.6706,0.1392
1,5,0.0000,0.4194,0.0000,0.2765,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.1373,1.1186,0.4230,1.2730,0.6045,0.0000,0.0000,0.0000,0.0000
2,7,0.9382,0.0000,0.0000,0.9751,0.0000,0.4528,0.0000,0.0000,0.2808,...,0.0000,0.0000,0.0000,0.4547,0.0979,0.0000,0.0000,0.0000,0.0000,0.0000
3,8,0.0777,0.9384,0.3952,0.0000,0.0000,0.0000,0.0000,1.2670,1.0444,...,0.3379,0.2252,0.4315,0.0759,2.4016,1.0923,0.0000,0.0000,0.7912,0.0000
4,9,1.5701,0.0000,1.2451,0.5674,1.2224,0.0000,0.0000,0.0000,0.1774,...,0.2465,0.0000,1.6743,0.0000,0.0409,0.8993,0.0000,1.2093,0.0000,1.0428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,247,0.9274,0.4696,0.0000,0.0000,0.0000,0.1226,1.1528,0.0000,0.2090,...,0.0000,0.0000,1.0360,1.2922,0.4502,0.0000,0.0000,0.0000,0.0000,0.1357
168,250,0.3749,0.0000,0.0000,0.4142,1.8517,0.0000,0.0323,0.1412,0.0000,...,0.1832,0.0000,0.0000,0.0130,0.0000,0.0000,0.2867,0.0000,0.0000,0.0000
169,251,0.1518,0.0000,0.0000,0.0000,0.4617,0.4079,0.0000,0.0000,0.0851,...,0.8342,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.2120,0.0000
170,252,0.0000,0.0000,0.0000,0.0101,0.0000,0.0000,0.0000,0.5485,1.0947,...,0.0000,0.6041,0.0242,0.0000,0.0747,0.3986,0.0000,0.0000,0.0000,0.0000


In [ ]:
df_x = pd.merge(df_seq, features2, on='video')
df_x = df_x.drop('video', axis = 1)  #drop video column

In [ ]:
print(df_x)

     0_x  1_x  2_x  3_x  4_x  5_x  6_x  7_x  8_x  9_x  ...  4086_y  4087_y  \
0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0000  0.4631   
1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.2485  0.2706   
2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.2953  0.0000   
3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0000  0.6871   
4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0000  0.1177   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...     ...     ...   
167  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.3073  0.0000   
168  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0000  0.0000   
169  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0000  0.0000   
170  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.1887  0.0417   
171  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0000  0.0000   

     4088_y  4089_y  4090_y  4091_y  4092_y  4093_y  4094_y  40

In [ ]:
labels

,video,mem_scores
0,17,0.989151
1,3628,0.913391
2,6515,0.932910
3,471,0.966062
4,4171,0.944808
...,...,...
6995,7054,0.844986
6996,4248,0.864480
6997,2031,0.783653
6998,8158,0.835309


In [ ]:
labels['mem_scores'].values

array([0.98915145, 0.91339093, 0.93291037, ..., 0.78365294, 0.83530909,
       0.76333768])

In [ ]:
Y = []

for filename in os.listdir('drive/MyDrive/C3D/C3D'):
  for c, v in labels.iterrows():
    # print(v['video'], int(i))
    if int(v['video']) == int(filename[:5]):
      print(v)
      Y.append(v['mem_scores'])
    # break

video              1
mem_scores    0.9451
Name: 645, dtype: object
video                5
mem_scores    0.830044
Name: 2485, dtype: object
video                7
mem_scores    0.796515
Name: 4011, dtype: object
video                8
mem_scores    0.940524
Name: 5504, dtype: object
video                9
mem_scores    0.979037
Name: 3552, dtype: object
video               10
mem_scores    0.775867
Name: 4166, dtype: object
video               11
mem_scores    0.912158
Name: 4068, dtype: object
video              13
mem_scores    0.78278
Name: 2537, dtype: object
video               14
mem_scores    0.822483
Name: 257, dtype: object
video               15
mem_scores    0.527761
Name: 2085, dtype: object
video               16
mem_scores    0.644263
Name: 212, dtype: object
video               17
mem_scores    0.989151
Name: 0, dtype: object
video               18
mem_scores    0.978537
Name: 3880, dtype: object
video               19
mem_scores    0.816809
Name: 6738, dtype: object
vide

In [ ]:
Y = np.array(Y)

In [ ]:
X = df_x.values

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [ ]:
Y_train

array([0.80893223, 0.64225693, 0.69703405, 0.78305492, 0.82211864,
       0.89879748, 0.79651534, 0.84194905, 0.8182456 , 0.86308941,
       0.8221352 , 0.62940641, 0.62359519, 0.67150378, 0.68037193,
       0.73862886, 0.82170905, 0.64180216, 0.61440586, 0.80460219,
       0.79362384, 0.79569758, 0.95601098, 0.62590387, 0.98915145,
       0.96004323, 0.85595502, 0.9121583 , 0.93582926, 0.61837072,
       0.97903653, 0.73204669, 0.84268242, 0.83437417, 0.64758284,
       0.64426259, 0.90884369, 0.54213659, 0.94510029, 0.94496661,
       0.77435565, 0.88757443, 0.9584034 , 0.82599898, 0.64921135,
       0.85905827, 0.84921201, 0.56819278, 0.57273212, 0.95618709,
       0.76226125, 0.61119895, 0.92475907, 0.89701432, 0.58594265,
       0.81462787, 0.80568488, 0.58563665, 0.85763688, 0.84774543,
       0.65319799, 0.9545717 , 0.82075285, 0.65505324, 0.8914459 ,
       0.87928079, 0.77651741, 0.57085957, 0.80904621, 0.84158334,
       0.67370817, 0.72549738, 0.79254353, 0.86925829, 0.77586

In [ ]:
print('X_train', X_train.shape)
print('X_test', X_test.shape)
print('Y_train', Y_train.shape)
print('Y_test', Y_test.shape)

X_train (137, 14117)
X_test (35, 14117)
Y_train (137,)
Y_test (35,)


In [ ]:
from sklearn import ensemble

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor(verbose=True)
RFregressor.fit(X_train, Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   15.8s finished


RandomForestRegressor(verbose=True)

In [ ]:
y_pred = RFregressor.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
stats.spearmanr(y_pred, Y_test)

SpearmanrResult(correlation=-0.1112044817927171, pvalue=0.5247901291372079)